In [27]:
from openai import OpenAI
import yaml
from models import Task
import os

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

def load_config(file_path):
    with open(file_path,'r') as file:
        return yaml.safe_load(file)

config=load_config('config.yaml')
tools=config['openai_tool']

In [9]:
tools

{'tool1': {'messages': [{'role': 'system',
    'content': 'Categorize the task into one of the following: Work, Personal, Errand, Study, Health, Other.'},
   {'role': 'user',
    'content': 'Task Title: {title}\nDescription: {description}'}],
  'model': 'gpt-4o-mini',
  'temperature': 0.6,
  'max_tokens': 40},
 'tool2': {'messages': [{'role': 'system', 'content': 'Summarize the task.'},
   {'role': 'user',
    'content': 'Task Title: {title}\nDescription: {description}'}],
  'model': 'gpt-4o-mini',
  'temperature': 1.0,
  'max_tokens': 100}}

In [18]:
tools["tool1"]

{'messages': [{'role': 'system',
   'content': 'Categorize the task into one of the following: Work, Personal, Errand, Study, Health, Other.'},
  {'role': 'user',
   'content': 'Task Title: {title}\nDescription: {description}\n'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.6,
 'max_tokens': 40}

In [48]:
title = "Finish Assignment"
description = "Complete the math assignment by tonight."

In [49]:
tool1 = config["openai_tool"]["tool1"]["messages"][1]["content"].format(title=title, description=description)

In [50]:
tools

{'tool1': {'messages': [{'role': 'system',
    'content': 'Categorize the task into one of the following: Work, Personal, Errand, Study, Health, Other.'},
   {'role': 'user',
    'content': 'Task Title: {title}\nDescription: {description}\n'}],
  'model': 'gpt-4o-mini',
  'temperature': 0.6,
  'max_tokens': 40},
 'tool2': {'messages': [{'role': 'system', 'content': 'Summarize the task.'},
   {'role': 'user',
    'content': 'Task Title: {title}\nDescription: {description}\n'}],
  'model': 'gpt-4o-mini',
  'temperature': 1.0,
  'max_tokens': 100}}

In [68]:
response = client.chat.completions.create(
    model=tools["tool1"]["model"],
    messages=[
        {"role": "system", "content": tools["tool1"]["messages"][0]["content"]},
        {"role": "user", "content": "Task Title: dota 2\nDescription: play dota with boys at 9pm"}
    ],
    temperature=tools["tool1"]["temperature"],
    max_tokens=tools["tool1"]["max_tokens"]
)
answer = response.choices[0].message.content.strip()
print(answer)


Category: Personal


In [99]:
response.choices[0].message.content

'Category: Personal'

In [ ]:
from firebase_admin import firestore,credentials
import firebase_admin
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()
task_ref=db.collection('tasks')

In [9]:
db

In [14]:
id_ref = db.collection("tasks").document("gVvoyJy5ZQRSdLAEHrUa")
collections = id_ref.collections()
for collection in collections:
    for doc in collection.stream():
        print(f"{doc.id} => {doc.to_dict()}")

In [18]:
doc_ref = db.collection("tasks").document("gVvoyJy5ZQRSdLAEHrUa")

doc = doc_ref.get()
if doc.exists:
    print(f"Document data: {doc.to_dict()}")
else:
    print("No such document!")

Document data: {'created_at': DatetimeWithNanoseconds(2024, 12, 22, 17, 26, tzinfo=datetime.timezone.utc), 'title': 'Checkup', 'description': 'Need to take my mother to the hospital for her daily checkup.', 'summary': None, 'category': 'Health'}


In [7]:
city_ref